### Imort something

In [ ]:
! pip install joblib==1.1.0

In [ ]:
#Check GPU version
! /usr/local/cuda/bin/nvcc --version
! nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import urllib.request

# URLs for the zip files
links = [
  'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
  'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
  'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
	'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
  'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
	'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
	'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
  'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
	'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
	'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
	'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
	'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
]

for idx, link in enumerate(links):
    fn = 'images_%02d.tar.gz' % (idx+1)
    print('downloading'+fn+'...')
    urllib.request.urlretrieve(link, fn)  # download the zip file

In [ ]:
#Make dictionary to save data
#! mkdir /content/drive/MyDrive/ML
#! mkdir /content/drive/MyDrive/ML/data
#! mkdir /content/drive/MyDrive/ML/history_model
#! mkdir /content/drive/MyDrive/ML/data/images
#! mkdir /content/drive/MyDrive/ML/history_model/run_dir

In [ ]:
#Unzip file
! tar -xzvf images_01.tar.gz -C /content
! rm images_01.tar.gz
! tar -xzvf images_02.tar.gz -C /content
! rm images_02.tar.gz
! tar -xzvf images_03.tar.gz -C /content
! rm images_03.tar.gz
! tar -xzvf images_04.tar.gz -C /content
! rm images_04.tar.gz
! tar -xzvf images_05.tar.gz -C /content
! rm images_05.tar.gz
! tar -xzvf images_06.tar.gz -C /content
! rm images_06.tar.gz
! tar -xzvf images_07.tar.gz -C /content
! rm images_07.tar.gz
! tar -xzvf images_08.tar.gz -C /content
! rm images_08.tar.gz
! tar -xzvf images_09.tar.gz -C /content
! rm images_09.tar.gz
! tar -xzvf images_10.tar.gz -C /content
! rm images_10.tar.gz
! tar -xzvf images_11.tar.gz -C /content
! rm images_11.tar.gz
! tar -xzvf images_12.tar.gz -C /content
! rm images_12.tar.gz

In [ ]:
from __future__ import division
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import os
import sys
import pprint
import warnings
import json
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision.models import densenet121 as torch_densenet121

#from densenet import densenet121
#import util

### DenseNet

In [ ]:
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from collections import OrderedDict

__all__ = ['DenseNet', 'densenet121', 'densenet169', 'densenet201', 'densenet161']


model_urls = {
    'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',
    'densenet169': 'https://download.pytorch.org/models/densenet169-b2777c0a.pth',
    'densenet201': 'https://download.pytorch.org/models/densenet201-c1103571.pth',
    'densenet161': 'https://download.pytorch.org/models/densenet161-8d451a50.pth',
}


def densenet121(pretrained=False, **kwargs):
    r"""Densenet-121 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    #model = DenseNet(num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16), **kwargs)
    model = torch_densenet121(weights="IMAGENET1K_V1" if pretrained else None)  # 使用 PyTorch 預訓練的模型
    """
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['densenet121']))
    """
    return model


def densenet169(pretrained=False, **kwargs):
    r"""Densenet-169 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = DenseNet(num_init_features=64, growth_rate=32, block_config=(6, 12, 32, 32),
                     **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['densenet169']))
    return model


def densenet201(pretrained=False, **kwargs):
    r"""Densenet-201 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = DenseNet(num_init_features=64, growth_rate=32, block_config=(6, 12, 48, 32),
                     **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['densenet201']))
    return model


def densenet161(pretrained=False, **kwargs):
    r"""Densenet-161 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = DenseNet(num_init_features=96, growth_rate=48, block_config=(6, 12, 36, 24),
                     **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['densenet161']))
    return model


class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm.1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu.1', nn.ReLU(inplace=True)),
        self.add_module('conv.1', nn.Conv2d(num_input_features, bn_size *
                        growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm.2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu.2', nn.ReLU(inplace=True)),
        self.add_module('conv.2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                        kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=1000):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        self.gap = nn.AdaptiveAvgPool2d(1)

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = self.gap(out).view(features.size(0), -1)
        out = self.classifier(out)
        return out

### util

In [ ]:
import pandas as pd
import torch.utils.data as data
import torchvision.transforms as transforms
import sklearn.metrics
import argparse
import os

from torch.autograd import Variable
from PIL import Image


def get_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", default="densenet", type=str)
    parser.add_argument("--optimizer", default="adam", type=str)
    parser.add_argument("--lr", default=0.0001, type=float)
    parser.add_argument("--weight_decay", default=0.0, type=float)
    parser.add_argument("--drop_rate", default=0.0, type=float)
    parser.add_argument("--epochs", default=15, type=int)
    parser.add_argument("--batch_size", default=16, type=int)
    parser.add_argument("--workers", default=8, type=int)
    parser.add_argument("--seed", default=123456, type=int)
    parser.add_argument("--tag", default="", type=str)
    parser.add_argument("--toy", action="store_true")
    parser.add_argument("--save_path", default=None, type=str)
    parser.add_argument("--scale", default=224, type=int)
    parser.add_argument("--horizontal_flip", action="store_true")
    parser.add_argument("--verbose", action="store_true")
    parser.add_argument("--scratch", action="store_true")
    parser.add_argument("--train_weighted", action="store_true")
    parser.add_argument("--valid_weighted", action="store_true")
    parser.add_argument("--size", default=None, type=str)
    return parser



class Dataset(data.Dataset):
    def __init__(self, args, data_split):
        super(Dataset, self).__init__()

        if args.tag:
            tag = "_"+args.tag
            df = pd.read_csv(os.path.join("/content/drive/MyDrive/ML/data/%s%s.csv" % (data_split, tag)))
        else:
            df = pd.read_csv(os.path.join("/content/drive/MyDrive/ML/data/%s.csv" % (data_split)))

        if args.toy:
            df = df.sample(frac=0.01)

        self.df = df
        self.img_paths = df["Path"].tolist()
        self.pathologies = [col for col in df.columns.values if col != "Path"]

        self.labels = df[self.pathologies].values.astype(int)

        self.n_classes = self.labels.shape[1]

        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        if data_split == "train":
            transforms_lst = [
                transforms.Resize((args.scale, args.scale)),
                transforms.RandomHorizontalFlip() if args.horizontal_flip else None,
                transforms.ToTensor(),
                normalize,
            ]
            self.transform = transforms.Compose([t for t in transforms_lst if t])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((args.scale, args.scale)),
                transforms.ToTensor(),
                normalize,
            ])
        self.df = df


        if (data_split == "train" and args.train_weighted) or (data_split == "valid" and args.valid_weighted):
            self.get_weights(args, data_split)


    def get_weights(self, args, data_split):

        self.use_gpu = torch.cuda.is_available()
        p_count = (self.labels == 1).sum(axis = 0)
        self.p_count = p_count
        n_count = (self.labels == 0).sum(axis = 0)
        total = p_count + n_count

        # invert *opposite* weights to obtain weighted loss
        # (positives weighted higher, all weights same across batches, and p_weight + n_weight == 1)
        p_weight = n_count / total
        n_weight = p_count / total

        self.p_weight_loss = Variable(torch.FloatTensor(p_weight), requires_grad=False)
        self.n_weight_loss = Variable(torch.FloatTensor(n_weight), requires_grad=False)

        print ("Positive %s Loss weight:" % data_split, self.p_weight_loss.data.numpy())
        print ("Negative %s Loss weight:" % data_split, self.n_weight_loss.data.numpy())
        random_loss = sum((p_weight[i] * p_count[i] + n_weight[i] * n_count[i]) *\
                                               -np.log(0.5) / total[i] for i in range(self.n_classes)) / self.n_classes
        print ("Random %s Loss:" % data_split, random_loss)


    def __getitem__(self, index):
        img = Image.open(self.img_paths[index]).convert("RGB")
        label = self.labels[index]

        return self.transform(img), torch.LongTensor(label)

    def __len__(self):
        return len(self.img_paths)

    def weighted_loss(self, preds, target, epoch=1):

        weights = target.type(torch.FloatTensor) * (self.p_weight_loss.expand_as(target)) + \
                  (target == 0).type(torch.FloatTensor) * (self.n_weight_loss.expand_as(target))
        if self.use_gpu:
            weights = weights.cuda()
        loss = 0.0
        for i in range(self.n_classes):
            loss += nn.functional.binary_cross_entropy_with_logits(preds[:,i], target[:,i], weight=weights[:,i])
        return loss / self.n_classes


def evaluate(gts, probabilities, pathologies, use_only_index = None):
    assert(np.all(probabilities >= 0) == True)
    assert(np.all(probabilities <= 1) == True)

    def compute_metrics_for_class(i):
         p, r, t = sklearn.metrics.precision_recall_curve(gts[:, i], probabilities[:, i])
         PR_AUC = sklearn.metrics.auc(r, p)
         ROC_AUC = sklearn.metrics.roc_auc_score(gts[:, i], probabilities[:, i])
         F1 = sklearn.metrics.f1_score(gts[:, i], preds[:, i])
         acc = sklearn.metrics.accuracy_score(gts[:, i], preds[:, i])
         count = np.sum(gts[:, i])
         return PR_AUC, ROC_AUC, F1, acc, count

    PR_AUCs = []
    ROC_AUCs = []
    F1s = []
    accs = []
    counts = []
    preds = probabilities >= 0.5

    classes = [use_only_index] if use_only_index is not None else range(len(gts[0]))

    for i in classes:
        try:
            PR_AUC, ROC_AUC, F1, acc, count = compute_metrics_for_class(i)
        except ValueError:
            continue
        PR_AUCs.append(PR_AUC)
        ROC_AUCs.append(ROC_AUC)
        F1s.append(F1)
        accs.append(acc)
        counts.append(count)
        print('Class: {!s} Count: {:d} PR AUC: {:.4f} ROC AUC: {:.4f} F1: {:.3f} Acc: {:.3f}'.format(pathologies[i], count, PR_AUC, ROC_AUC, F1, acc))

    avg_PR_AUC = np.average(PR_AUCs)
    avg_ROC_AUC = np.average(ROC_AUCs, weights=counts)
    avg_F1 = np.average(F1s, weights=counts)

    print('Avg PR AUC: {:.3f}'.format(avg_PR_AUC))
    print('Avg ROC AUC: {:.3f}'.format(avg_ROC_AUC))
    print('Avg F1: {:.3f}'.format(avg_F1))
    return avg_PR_AUC, avg_ROC_AUC, avg_F1


def loader_to_gts(data_loader):
    gts = []
    for (inputs, labels) in data_loader:
        for label in labels.cpu().numpy().tolist():
            gts.append(label)
    gts = np.array(gts)
    return gts


def load_data(args):

    train_dataset = Dataset(args, "train_F")
    valid_dataset = Dataset(args, "valid_female")

    train_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=args.batch_size, shuffle=True,
            num_workers=args.workers, pin_memory=True, sampler=None)

    valid_loader = torch.utils.data.DataLoader(
            valid_dataset, batch_size=args.batch_size, shuffle=False,
            num_workers=args.workers, pin_memory=True, sampler=None)

    return train_loader, valid_loader

### train

In [ ]:
import matplotlib.pyplot as plt
import time
class DenseNet_train(nn.Module):
    def __init__(self, config, nclasses):
        super(DenseNet_train, self).__init__()
        self.model_ft = densenet121(pretrained=not config.scratch, drop_rate=config.drop_rate)
        num_ftrs = self.model_ft.classifier.in_features
        self.model_ft.classifier = nn.Linear(num_ftrs, nclasses)
        self.config = config
    def forward(self, x):
        return self.model_ft(x)

def transform_data(data, use_gpu, train=False):
    inputs, labels = data
    # GPU
    labels = labels.type(torch.FloatTensor)
    if use_gpu is True:
        inputs = inputs.cuda()
        labels = labels.cuda()
    #
    with torch.no_grad():
        inputs = Variable(inputs, requires_grad=False)
        labels = Variable(labels, requires_grad=False)
    return inputs, labels

In [ ]:
def train_epoch(epoch, args, model, loader, criterion, optimizer):
    model.train()
    batch_losses = []
    start_time = time.time() #記錄初始時間

    for batch_idx, data in enumerate(loader):
        batch_start_time = time.time()  #每個batch開始時間
        inputs, labels = transform_data(data, True, train=True)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels, epoch=epoch)
        loss.backward()
        optimizer.step()

        #計算累計時間
        cumulative_time = time.time() - start_time
        print("Epoch: {:d} Batch: {:d} ({:d}) Train Loss: {:.6f} Time Elapsed: {:.2f} seconds".format(
            epoch, batch_idx, args.batch_size, loss.data, cumulative_time))
        sys.stdout.flush()

        #batch_losses.append(loss.data) #CPU
        batch_losses.append(loss.data.cpu().numpy()) #GPU
    train_loss = np.mean(batch_losses)
    print("Training Loss: {:.6f}".format(train_loss))
    return train_loss


def test_epoch(model, loader, criterion, epoch=1):
    """
    Returns: (AUC, ROC AUC, F1, validation loss)
    """
    model.eval()
    test_losses = []
    outs = []
    gts = []
    for data in loader:
        #for gt in data[1].numpy().tolist(): #CPU
        for gt in data[1].cpu().numpy().tolist(): #GPU
            gts.append(gt)
        inputs, labels = transform_data(data, True, train=False)
        outputs = model(inputs)
        loss = criterion(outputs, labels, epoch=epoch)
        #test_losses.append(loss.data) #CPU
        test_losses.append(loss.data.cpu().numpy()) #GPU
        out = torch.sigmoid(outputs).data.cpu().numpy()
        outs.extend(out)
    avg_loss = np.mean(test_losses)
    print("Validation Loss: {:.6f}".format(avg_loss))
    outs = np.array(outs)
    gts = np.array(gts)
    avg_pr_auc, avg_roc_auc, avg_f1 = evaluate(gts, outs, loader.dataset.pathologies)
    print(f"Validation AUC: PR AUC={avg_pr_auc:.3f}, ROC AUC={avg_roc_auc:.3f}, F1 Score={avg_f1:.3f}")
    return evaluate(gts, outs, loader.dataset.pathologies) + (avg_loss,)


def get_loss(dataset, weighted):

    criterion = nn.MultiLabelSoftMarginLoss()

    def loss(preds, target, epoch):

        if weighted:

            return dataset.weighted_loss(preds, target, epoch=epoch)

        else:

            return criterion(preds, target)

    return loss


def run(args):

    use_gpu = torch.cuda.is_available()
    model = None

    train, val = load_data(args)
    nclasses = train.dataset.n_classes
    print("Number of classes:", nclasses)

    if args.model == "densenet":
        model = DenseNet_train(args, nclasses)
    else:
        print("{} is not a valid model.".format(args.model))

    if use_gpu:
        model = model.cuda()

    train_criterion = get_loss(train.dataset, args.train_weighted)
    val_criterion = get_loss(val.dataset, args.valid_weighted)

    if args.optimizer == "adam":
        optimizer = optim.Adam(
                       filter(lambda p: p.requires_grad, model.model_ft.parameters()),
                       lr=args.lr,
                       weight_decay=args.weight_decay)
    elif args.optimizer == "rmsprop":
        optimizer = optim.RMSprop(
                       filter(lambda p: p.requires_grad, model.model_ft.parameters()),
                       lr=args.lr,
                       weight_decay=args.weight_decay)
    else:
        print("{} is not a valid optimizer.".format(args.optimizer))

    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, threshold=0.001, factor=0.1)
    best_model_wts, best_loss = model.state_dict(), float("inf")

    #畫loss、AUC圖
    train_losses = []
    valid_losses = []
    roc_aucs = []
    elapsed_times = []
    start_time = time.time()

    plt.ion()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1.set_title("Training and Validation Loss Over Time")
    ax1.set_xlabel("Elapsed Time (s)")
    ax1.set_ylabel("Loss")
    ax2.set_title("Validation ROC AUC Over Time")
    ax2.set_xlabel("Elapsed Time (s)")
    ax2.set_ylabel("ROC AUC")

    ax1.plot([], [], label="Train Loss", color="blue")
    ax1.plot([], [], label="Validation Loss", color="orange")
    ax1.legend(loc='upper right')

    ax2.plot([], [], label="ROC AUC", color="green")
    ax2.legend(loc='upper right')


    counter = 0
    for epoch in range(1, args.epochs + 1):
        print("Epoch {}/{}".format(epoch, args.epochs))
        print("-" * 10)

        train_loss = train_epoch(epoch, args, model, train,train_criterion, optimizer)
        #picture
        train_losses.append(train_loss)

        _, epoch_auc, _, valid_loss = test_epoch(model, val, val_criterion, epoch)
        #picture
        valid_losses.append(valid_loss)
        roc_aucs.append(epoch_auc)
        elapsed_time = time.time() - start_time
        elapsed_times.append(elapsed_time)


        scheduler.step(valid_loss)

        if (valid_loss < best_loss):
            best_loss = valid_loss
            best_model_wts = model.state_dict()
            counter = 0
        else:
            counter += 1

        if counter > 3:
            break

        #即時更新圖
        ax1.plot(elapsed_times, train_losses, label="Train Loss", color="blue")
        ax1.plot(elapsed_times, valid_losses, label="Validation Loss", color="orange")

        ax2.plot(elapsed_times, roc_aucs, label="ROC AUC", color="green")

        fig.canvas.draw()
        plt.pause(0.01)

        torch.save(best_model_wts, os.path.join(args.save_path, "val%f_train%f_epoch%d" % (valid_loss, train_loss, epoch)))

    plt.ioff()
    plt.show()
    print("Best Validation Loss:", best_loss)

if __name__ == "__main__":
    """
    Usage
        Download the images data at https://nihcc.app.box.com/v/ChestXray-NIHCC
        To train on the original labels:
            python train.py --save_path run_dir --model densenet --batch_size 8 --horizontal_flip --epochs 10 --lr 0.0001 --train_weighted --valid_weighted --scale 512
        To train on the relabels:
            python train.py --save_path run_dir --model densenet --batch_size 8 --horizontal_flip --epochs 10 --lr 0.0001 --train_weighted --valid_weighted --scale 512 --tag relabeled
    """

    args = argparse.Namespace(
        model="densenet",
        optimizer="adam",
        lr=0.0001,
        weight_decay=0.0,
        drop_rate=0.0,
        epochs=10,
        batch_size=16,
        workers=4,
        seed = 123456,
        tag = "",
        toy = False,
        save_path="drive/MyDrive/ML/history_model/run_dir",  # 提供一個默認的保存路徑
        scale=224,
        horizontal_flip = False,
        verbose = False,
        scratch=False,  # 控制是否從頭開始訓練
        train_weighted=False,  # 是否加權訓練損失
        valid_weighted=False,  # 是否加權驗證損失
        size = None
    )
    """
    parser = get_parser()
    args = parser.parse_args()
    pp = pprint.PrettyPrinter()
    pp.pprint(vars(args))
    """

    if not os.path.isdir(args.save_path):
        os.makedirs(args.save_path)

    with open(os.path.join(args.save_path, "params.txt"), 'w') as out:
        json.dump(vars(args), out, indent=4)
    run(args)

### Predict

In [ ]:
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/female
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/female/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/male
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/male/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/young
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/young/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/middle
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/middle/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/old
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/old/predictions

# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/female
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/female/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/male
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/male/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/young
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/young/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/middle
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/middle/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/old
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/old/predictions

# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/female
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/female/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/male
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/male/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/young
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/young/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/middle
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/middle/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/old
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/old/predictions

# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/female
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/female/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/male
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/male/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/young
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/young/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/middle
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/middle/predictions
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/old
# ! mkdir /content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/old/predictions

In [ ]:
from __future__ import division
from __future__ import print_function
import numpy as np
import torch
import time
import os
import json
import argparse
import itertools
from sklearn import metrics
#import util
#from train import transform_data, DenseNet
from joblib import Memory


memory = Memory(location='./cache', verbose=0)


# convert dictionary to object
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)


def load_model(args, load_path, nclasses):
    #print(args)
    print(nclasses)
    model = DenseNet_train(args, nclasses)
    #model.load_state_dict(torch.load(load_path))
    model.load_state_dict(torch.load(load_path))
    model = model.cuda()
    model.eval()
    return model


@memory.cache
def get_model_predictions(args_dict, model_path, loader):

    args = Struct(**args_dict)

    model = load_model(args, model_path, loader.dataset.n_classes)
    outs = []
    # gather predictions for all images in the validation set
    for i, (inputs, labels) in enumerate(loader):
        inputs, _ = transform_data((inputs, labels), use_gpu=True)
        outputs = model(inputs)
        out = torch.sigmoid(outputs).data.cpu().numpy()
        outs.append(out)
    outs = np.concatenate(outs, axis=0)
    return outs


@memory.cache
def get_loader_on_split(args_dict, split):

    if "tag" not in args_dict:
        args_dict["tag"] = args_dict["size"]

    args = Struct(**args_dict)

    dataset = Dataset(args, split)

    loader = torch.utils.data.DataLoader(
                    dataset, batch_size=args.batch_size, shuffle=False,
                    num_workers=args.workers, pin_memory=False)
    return loader


def optimal_threshold_compute(labels, probs):
    thresholds = []
    for i in range(probs.shape[1]):
        p, r, t = metrics.precision_recall_curve(labels[:, i], probs[:, i])
        f1_scores = np.where((p + r) == 0, 0, 2 * p * r / (p + r)) #防止/0
        threshold = t[np.nanargmax(f1_scores)]
        thresholds.append(threshold)
    thresholds = np.array(thresholds)
    print("Optimal thresholds at ", thresholds)
    return thresholds

def predict_for_split(args_dicts, model_paths, split):
    loaders = [get_loader_on_split(args_dict, split) for args_dict in args_dicts]
    all_model_probs = [get_model_predictions(args_dict, model_path, loader) \
            for args_dict, model_path, loader in zip(args_dicts, model_paths, loaders)]
    probs = np.mean(all_model_probs, axis=0)
    # labels should be the same across all models
    labels = loaders[0].dataset.labels

    if 'valid' in split:
        thresholds = optimal_threshold_compute(labels, probs)
        #print(loaders[0].dataset.pathologies)
        auc = metrics.roc_auc_score(labels, probs)
        print("AUC", auc)
        name = str(auc) + '-' + split
    else:
        name = split
        thresholds = None

    return probs, thresholds, name


def predict(model_paths, split="valid_young", save=True):

    def get_model_params(model_path):
        #params = json.load(open(
        #    os.path.dirname(model_path) + '/params.txt', 'r'))
        params = json.load(open('/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/params.txt', 'r'))
        return params

    model_args_dicts = [get_model_params(model_path) for model_path in model_paths]

    folder_name = '/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/female/predictions' + str(int(round(time.time() * 1000)))

    probs, thresholds, name = predict_for_split(model_args_dicts, model_paths, split)

    if save:
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        np.save(folder_name + '/' + name, probs)

        try:
            loaded_probs = np.load(folder_name + '/' + name + ".npy", allow_pickle=True)
            print("Loaded probs shape:", loaded_probs.shape)
        except Exception as e:
            print("Error loading .npy file:", e)

        print("Predictions saved to ", folder_name)

        params = {"model_paths": model_paths,
                  "num_models" : len(model_paths)}
        with open(folder_name + '/params.json', 'w') as outfile:
            json.dump(params, outfile)

        return folder_name + '/' + name + ".npy"

    else:

        return probs, thresholds


if __name__ == "__main__":
    """
    Usage
        python predict.py model1 model2 ... modelN
    """

    """
    parser = util.get_parser()
    parser.add_argument(
        'model_paths',
        nargs='+',
        help="path to models")
    args = parser.parse_args()
    """

    args = argparse.Namespace(
        model_paths=["/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.276692_train0.304059_epoch1", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.262023_train0.267144_epoch2", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.237520_train0.243476_epoch3", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.228785_train0.211695_epoch4", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.205620_train0.163654_epoch5", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.215539_train0.109086_epoch6", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.220947_train0.071413_epoch7", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.187284_train0.035033_epoch8", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.188101_train0.021869_epoch9", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.193619_train0.015704_epoch10"])
    predict(args.model_paths, save=False)

預測驗證集中每個病例在各病徵的機率

In [ ]:
import numpy as np

file_path = '/content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/middle/predictions/1732349921323/0.8196511379037413-valid_middle.npy'
try:
    data = np.load("/content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/middle/predictions/1732349921323/0.8196511379037413-valid_middle.npy")
    print(data)
except Exception as e:
    print("Error loading .npy file:", e)


預測疾病

In [ ]:
import torch
import numpy as np
import pandas as pd
from torchvision import transforms
from PIL import Image
from torch.autograd import Variable

# 定義模型載入函數
def load_model(weight_path, num_classes=4):
    # 初始化模型時手動提供必要的參數
    class Config:
        scratch = False
        drop_rate = 0.0
    config = Config()

    model = DenseNet_train(config=config, nclasses=num_classes)  # 使用 Config 提供必要參數
    model.load_state_dict(torch.load(weight_path))
    model.eval()
    model = model.cuda()
    return model

# 定義影像預處理函數
def preprocess_image(image_path, scale=224):
    transform = transforms.Compose([
        transforms.Resize((scale, scale)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0)

def predict(models, image_paths, thresholds):
    """
    使用多個模型對圖像進行預測，並應用對應病徵的 threshold 進行分類
    models: 模型列表
    image_paths: 圖像路徑列表
    thresholds: 各病徵的 threshold 值 (形狀: [num_classes])
    """
    all_predictions = []
    for image_path in image_paths:
        image = preprocess_image(image_path).cuda()
        predictions = []
        for model in models:
            with torch.no_grad():
                outputs = model(image)
                probabilities = torch.sigmoid(outputs).cpu().numpy()
                predictions.append(probabilities)
        avg_prediction = np.mean(predictions, axis=0)
        binary_prediction = (avg_prediction >= thresholds).astype(int)  # 根據 threshold 分類
        all_predictions.append(binary_prediction.flatten())
    return all_predictions

# 定義每個病徵的特定 threshold
thresholds = np.array([0.38284844, 0.46058702, 0.4600233, 0.4011343])  # 對應 [Nodule, Infiltration, Effusion, Atelectasis]


# 權重檔案路徑
weight_paths = [
    "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.276692_train0.304059_epoch1", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.262023_train0.267144_epoch2", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.237520_train0.243476_epoch3", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.228785_train0.211695_epoch4", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.205620_train0.163654_epoch5", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.215539_train0.109086_epoch6", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.220947_train0.071413_epoch7", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.187284_train0.035033_epoch8", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.188101_train0.021869_epoch9", "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/run_dir/val0.193619_train0.015704_epoch10"]

# 影像檔案路徑
image_paths = [
    #A
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000072_000.png",
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000086_002.png",
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000119_001.png",
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000120_000.png",
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000248_008.png",
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000307_000.png",
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000324_015.png",
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000421_000.png",
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000963_011.png",
    "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00001088_008.png",
]

# 載入所有模型
models = [load_model(weight_path) for weight_path in weight_paths]

# 預測並應用 threshold
predictions = predict(models, image_paths, thresholds=thresholds)

# 提取原始圖片編號作為索引
image_ids = [os.path.splitext(os.path.basename(path))[0] for path in image_paths]

# 建立表格
df = pd.DataFrame(predictions, columns=["Nodule", "Infiltration", "Effusion", "Atelectasis"], index=image_ids)

# 輸出結果
print(df)

# 儲存為 CSV
output_path = "/content/drive/MyDrive/Machine-learning-project/predictions_with_image_ids.csv"
df.to_csv(output_path, index=True)
print(f"Predictions saved to {output_path}")

**Model_1-ROC**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

csv_path = '/content/drive/MyDrive/ML/data/valid_F.csv'
df = pd.read_csv(csv_path)

true_labels = df.iloc[:, 1:].values

probs = np.load('/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/female/predictions/1732374997769/0.8291619762265146-valid_F.npy', allow_pickle=True)

class_names = df.columns[1:]

def plot_multiclass_roc(true_labels, probs, class_names):
    plt.figure(figsize=(10, 8))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(true_labels[:, i], probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

    #plot total roc curve
    #start
    fpr, tpr, _ = roc_curve(true_labels.reshape(-1), probs.reshape(-1))
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f"Total (AUC = {roc_auc:.2f})")
    #end

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("ROC AUC Curves for Train_female Each Symptom in Model_CheXneXt")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_multiclass_roc(true_labels, probs, class_names)


**Model_2-ROC**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

csv_path = '/content/drive/MyDrive/ML/data/valid_F.csv'
probs_csv_path = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_F_pre.csv'

df = pd.read_csv(csv_path)
true_labels = df.iloc[:, 1:].values
class_names = df.columns[1:]

probs = pd.read_csv(probs_csv_path).values

def plot_multiclass_roc(true_labels, probs, class_names):
    plt.figure(figsize=(10, 8))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(true_labels[:, i], probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

    fpr, tpr, _ = roc_curve(true_labels.ravel(), probs.ravel())
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f"Total (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("ROC AUC Curves for Train_female Each Symptom in Model_Xception")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_multiclass_roc(true_labels, probs, class_names)


10 條 ROC 曲線，分別對應 5 個客群的 Xception 模型 和 CheXneXt 模型

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# 路徑設定
data_paths = {
    "Female": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_F.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/female/predictions/1732374997769/0.8291619762265146-valid_F.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_F_pre.csv"
    },
    "Male": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_M.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/male/predictions/1732380203052/0.8276335029453044-valid_M.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_M_1119_2327/roc_M_pre.csv"
    },
    "Young": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_young.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/young/predictions/1732372765129/0.927858775074568-valid_young.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_young__1122_1701/roc_young_pre.csv"
    },
    "Middle": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_middle.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/middle/predictions/1732349921323/0.8196511379037413-valid_middle.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_middle__1122_2205/roc_middle_pre.csv"
    },
    "Old": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_old.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_age/old/ep_10/predict/old/predictions/1732285693842/0.8736757582492061-valid_old.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_old__1123_0057/roc_old_pre.csv"
    }
}

# 繪製多模型 ROC 曲線
def plot_10_model_roc(data_paths):
    plt.figure(figsize=(12, 10))

    # 顏色設定
    colors = {
        "Female": "blue",
        "Male": "green",
        "Young": "orange",
        "Middle": "purple",
        "Old": "red"
    }

    for group, paths in data_paths.items():
        # 讀取真實標籤
        df = pd.read_csv(paths["true_csv"])
        true_labels = df.iloc[:, 1:].values

        # 讀取 CheXneXt 預測值
        chexnext_probs = np.load(paths["chexnext_probs"], allow_pickle=True)

        # 讀取 Xception 預測值
        xception_probs = pd.read_csv(paths["xception_probs"]).values

        # 繪製 CheXneXt 的總 ROC 曲線（實線）
        fpr, tpr, _ = roc_curve(true_labels.ravel(), chexnext_probs.ravel())
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, linestyle='-', color=colors[group], label=f"CheXneXt {group} (AUC = {roc_auc:.2f})")

        # 繪製 Xception 的總 ROC 曲線（虛線）
        fpr, tpr, _ = roc_curve(true_labels.ravel(), xception_probs.ravel())
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, linestyle='--', color=colors[group], label=f"Xception {group} (AUC = {roc_auc:.2f})")

    # 添加圖例與參考線
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("TOTAL ROC AUC for CheXneXt and Xception Across Groups")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

# 繪製曲線
plot_10_model_roc(data_paths)


10個model的全預測線(每張圖5條)_Model1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

groups = {
    "Female": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_F.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/female/predictions/1732379111661/0.7556504091210221-valid_F.npy",
    },
    "Male": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_M.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/male/predictions/1732380203052/0.8276335029453044-valid_M.npy",
    },
    "Middle": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_middle.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/middle/predictions/1732381048158/0.7938877349467963-valid_middle.npy",
    },
    "Old": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_old.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/old/predictions/1732382123865/0.7663979892214088-valid_old.npy",
    },
    "Young": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_young.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/young/predictions/1732382952584/0.805614894052909-valid_young.npy",
    },
}

def plot_total_roc(groups):
    plt.figure(figsize=(10, 8))

    for group_name, paths in groups.items():
        df = pd.read_csv(paths["csv_path"])
        true_labels = df.iloc[:, 1:].values
        probs = np.load(paths["npy_path"], allow_pickle=True)

        fpr, tpr, _ = roc_curve(true_labels.ravel(), probs.ravel())
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f"{group_name} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("Total ROC AUC Curves for Train_male Groups")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_total_roc(groups)


10個model的全預測線(每張圖5條)_Model2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

groups = {
    "Female": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_F.csv",
        "start_column": 6,
    },
    "Male": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_M.csv",
        "start_column": 1,
    },
    "Middle": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_middle.csv",
        "start_column": 16,
    },
    "Old": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_old.csv",
        "start_column": 21,
    },
    "Young": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_young.csv",
        "start_column": 11,
    },
}

prediction_csv_path = "/content/drive/MyDrive/Machine-learning-project/Xception/1122_result/F/F_prediction.csv"

def plot_total_roc(groups, prediction_csv_path):
    plt.figure(figsize=(10, 8))

    pred_data = pd.read_csv(prediction_csv_path, skiprows=1)

    for group_name, paths in groups.items():
        true_data = pd.read_csv(paths["true_csv_path"])
        true_labels = true_data.iloc[:, 1:].values

        start_col = paths["start_column"]
        pred_probs = pred_data.iloc[:, start_col : start_col + 4].values

        min_samples = min(true_labels.shape[0], pred_probs.shape[0])
        true_labels = true_labels[:min_samples, :]
        pred_probs = pred_probs[:min_samples, :]

        fpr, tpr, _ = roc_curve(true_labels.ravel(), pred_probs.ravel())
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f"{group_name} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("Total ROC AUC Curves for Train_female Groups")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_total_roc(groups, prediction_csv_path)


10個model的全預測線(每張圖5條)_Model2_2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

groups = {
    "Female": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_F.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_old__1123_0057/roc_F_pre.csv",
    },
    "Male": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_M.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_old__1123_0057/roc_M_pre.csv",
    },
    "Middle": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_middle.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_old__1123_0057/roc_middle_pre.csv",
    },
    "Old": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_old.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_old__1123_0057/roc_old_pre.csv",
    },
    "Young": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_young.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_old__1123_0057/roc_young_pre.csv",
    },
}

def plot_total_roc(groups):
    plt.figure(figsize=(10, 8))

    for group_name, paths in groups.items():
        true_data = pd.read_csv(paths["true_csv_path"])
        true_labels = true_data.iloc[:, 1:].values

        pred_data = pd.read_csv(paths["pred_csv_path"], skiprows=0)
        pred_probs = pred_data.values

        min_samples = min(true_labels.shape[0], pred_probs.shape[0])
        true_labels = true_labels[:min_samples, :]
        pred_probs = pred_probs[:min_samples, :]

        fpr, tpr, _ = roc_curve(true_labels.ravel(), pred_probs.ravel())
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f"{group_name} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("Total ROC AUC Curves for Train_old Groups")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_total_roc(groups)


同客群不同model的比較(5張圖)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

csv_path = '/content/drive/MyDrive/ML/data/valid_F.csv'
chexnext_probs_path = '/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/female/predictions/1732374997769/0.8291619762265146-valid_F.npy'
xception_probs_path = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_F_pre.csv'

df = pd.read_csv(csv_path)
true_labels = df.iloc[:, 1:].values
class_names = df.columns[1:]

chexnext_probs = np.load(chexnext_probs_path, allow_pickle=True)
xception_probs = pd.read_csv(xception_probs_path).values

def plot_multiclass_roc(true_labels, chexnext_probs, xception_probs, class_names):
    plt.figure(figsize=(12, 10))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(true_labels[:, i], chexnext_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, linestyle='--', label=f"CheXneXt {class_names[i]} (AUC = {roc_auc:.2f})")

    fpr, tpr, _ = roc_curve(true_labels.ravel(), chexnext_probs.ravel())
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=3, linestyle='-', label=f"CheXneXt Total (AUC = {roc_auc:.2f})")

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(true_labels[:, i], xception_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, linestyle='--', label=f"Xception {class_names[i]} (AUC = {roc_auc:.2f})")

    fpr, tpr, _ = roc_curve(true_labels.ravel(), xception_probs.ravel())
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=3, linestyle='-', label=f"Xception Total (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("ROC AUC Curves for CheXneXt and Xception in female_Group")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_multiclass_roc(true_labels, chexnext_probs, xception_probs, class_names)


同Model_1不同客群的比較

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

groups = {
    "Female": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_F.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/female/predictions/1732374997769/0.8291619762265146-valid_F.npy",
    },
    "Male": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_M.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/male/predictions/1732380203052/0.8276335029453044-valid_M.npy",
    },
    "Middle": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_middle.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/middle/predictions/1732349921323/0.8196511379037413-valid_middle.npy",
    },
    "Old": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_old.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_age/old/ep_10/predict/old/predictions/1732285693842/0.8736757582492061-valid_old.npy",
    },
    "Young": {
        "csv_path": "/content/drive/MyDrive/ML/data/valid_young.csv",
        "npy_path": "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/young/predictions/1732372765129/0.927858775074568-valid_young.npy",
    },
}

def plot_total_roc(groups):
    plt.figure(figsize=(10, 8))

    for group_name, paths in groups.items():
        df = pd.read_csv(paths["csv_path"])
        true_labels = df.iloc[:, 1:].values
        probs = np.load(paths["npy_path"], allow_pickle=True)

        fpr, tpr, _ = roc_curve(true_labels.ravel(), probs.ravel())
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f"{group_name} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("Total ROC AUC Curves for Xception each Groups")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_total_roc(groups)


同Model_2不同客群的比較

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

groups = {
    "Female": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_F.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_F_pre.csv",
    },
    "Male": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_M.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_M_1119_2327/roc_M_pre.csv",
    },
    "Middle": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_middle.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_middle__1122_2205/roc_middle_pre.csv",
    },
    "Old": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_old.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_old__1123_0057/roc_old_pre.csv",
    },
    "Young": {
        "true_csv_path": "/content/drive/MyDrive/ML/data/valid_young.csv",
        "pred_csv_path": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_young__1122_1701/roc_young_pre.csv",
    },
}

def plot_total_roc(groups):
    plt.figure(figsize=(10, 8))

    for group_name, paths in groups.items():
        true_data = pd.read_csv(paths["true_csv_path"])
        true_labels = true_data.iloc[:, 1:].values

        pred_data = pd.read_csv(paths["pred_csv_path"], skiprows=0)
        pred_probs = pred_data.values

        min_samples = min(true_labels.shape[0], pred_probs.shape[0])
        true_labels = true_labels[:min_samples, :]
        pred_probs = pred_probs[:min_samples, :]

        fpr, tpr, _ = roc_curve(true_labels.ravel(), pred_probs.ravel())
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f"{group_name} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("Total ROC AUC Curves for Xception each Groups")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_total_roc(groups)


對於不同種客群兩個model在四個病徵中的比較

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

data_paths = {
    "Female": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_F.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Female/ep_10/predict/female/predictions/1732374997769/0.8291619762265146-valid_F.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_F_pre.csv"
    },
    "Male": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_M.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/predict/male/predictions/1732380203052/0.8276335029453044-valid_M.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_M_1119_2327/roc_M_pre.csv"
    },
    "Young": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_young.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/young/predictions/1732372765129/0.927858775074568-valid_young.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_young__1122_1701/roc_young_pre.csv"
    },
    "Middle": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_middle.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_age/middle/ep_10/predict/middle/predictions/1732349921323/0.8196511379037413-valid_middle.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_middle__1122_2205/roc_middle_pre.csv"
    },
    "Old": {
        "true_csv": "/content/drive/MyDrive/ML/data/valid_old.csv",
        "chexnext_probs": "/content/drive/MyDrive/ML/history_model/CheXneXt_age/old/ep_10/predict/old/predictions/1732285693842/0.8736757582492061-valid_old.npy",
        "xception_probs": "/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_old__1123_0057/roc_old_pre.csv"
    }
}

def plot_nodule_roc(data_paths):
    plt.figure(figsize=(12, 10))

    colors = {
        "Female": "blue",
        "Male": "green",
        "Young": "orange",
        "Middle": "purple",
        "Old": "red"
    }

    for group, paths in data_paths.items():
        df = pd.read_csv(paths["true_csv"])
        true_labels = df.iloc[:, 1:].values

        chexnext_probs = np.load(paths["chexnext_probs"], allow_pickle=True)
        xception_probs = pd.read_csv(paths["xception_probs"]).values

        fpr, tpr, _ = roc_curve(true_labels[:, 3], chexnext_probs[:, 3])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, linestyle='-', color=colors[group], label=f"CheXneXt {group} (AUC = {roc_auc:.2f})")
        fpr, tpr, _ = roc_curve(true_labels[:, 3], xception_probs[:, 3])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, linestyle='--', color=colors[group], label=f"Xception {group} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.50)')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("Atelectasis ROC AUC for CheXneXt and Xception")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

plot_nodule_roc(data_paths)


**grad cam**

In [ ]:
! pip install grad-cam

In [ ]:
import os
import torch
import numpy as np
from torchvision.models import densenet121
from torchvision.transforms import transforms
from PIL import Image
import matplotlib.pyplot as plt

def preprocess_image(image_path, target_size=(224, 224)):
    preprocess = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    return preprocess(image).unsqueeze(0)

def generate_grad_cam_pytorch(model, image_tensor, target_layer):
    model.eval()
    activations = []
    gradients = []

    def forward_hook(module, input, output):
        activations.append(output)

    def backward_hook(module, grad_in, grad_out):
        gradients.append(grad_out[0])

    handle_forward = target_layer.register_forward_hook(forward_hook)
    handle_backward = target_layer.register_backward_hook(backward_hook)

    output = model(image_tensor)
    target_class = torch.argmax(output, dim=1)
    loss = output[:, target_class]

    model.zero_grad()
    loss.backward()

    gradient = gradients[0].cpu().data.numpy()
    activation = activations[0].cpu().data.numpy()

    weights = np.mean(gradient, axis=(2, 3))
    cam = np.zeros(activation.shape[2:], dtype=np.float32)

    for i, w in enumerate(weights[0]):
        cam += w * activation[0, i, :, :]

    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)
    cam = np.uint8(255 * cam)

    handle_forward.remove()
    handle_backward.remove()

    return cam

def overlay_grad_cam_high_res(image_path, heatmap, alpha=0.4):
    original_img = Image.open(image_path).convert("RGB")
    original_size = original_img.size

    heatmap_resized = Image.fromarray(heatmap).resize(original_size, Image.BILINEAR)
    heatmap_resized = np.array(heatmap_resized)

    heatmap_color = plt.cm.jet(heatmap_resized / 255.0)[:, :, :3]
    heatmap_color = (heatmap_color * 255).astype(np.uint8)

    overlay = np.array(original_img) / 255.0
    overlay = (1 - alpha) * overlay + alpha * (heatmap_color / 255.0)
    overlay = (overlay * 255).astype(np.uint8)

    return overlay

def process_and_save_grad_cam(model, image_path, target_layer, save_path):
    image_tensor = preprocess_image(image_path)
    heatmap = generate_grad_cam_pytorch(model, image_tensor, target_layer)
    overlay = overlay_grad_cam_high_res(image_path, heatmap)

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    Image.fromarray(overlay).save(save_path)

    plt.figure(figsize=(10, 10))
    plt.imshow(overlay)
    plt.axis("off")
    plt.title("High-Resolution Grad-CAM Overlay")
    plt.show()

model_paths = [
    "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/run_dir/val0.315201_train0.347674_epoch1",
    "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/run_dir/val0.305339_train0.308090_epoch2",
    "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/run_dir/val0.306597_train0.281822_epoch3",
    "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/run_dir/val0.316875_train0.245489_epoch4",
    "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/run_dir/val0.300346_train0.157751_epoch5",
    "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/run_dir/val0.312949_train0.113909_epoch6",
    "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/run_dir/val0.329378_train0.086568_epoch7",
    "/content/drive/MyDrive/ML/history_model/CheXneXt_gender/Male/ep_10/run_dir/val0.329106_train0.066263_epoch8"
]

image_paths = {
    "Atelectasis": [
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000072_000.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000086_002.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000119_001.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000120_000.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000248_008.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000307_000.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000324_015.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000421_000.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00000963_011.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Atelectasis/00001088_008.png",
    ],
    "Effusion": [
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000041_006.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000071_007.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000116_008.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000116_011.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000127_004.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000250_008.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000506_012.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000572_001.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000591_014.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Effusion/00000848_000.png",
    ],
    "Healthy": [
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00000049_002.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00000547_007.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00000627_001.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00000646_007.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00000652_018.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00000756_005.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00000820_020.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00000857_001.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00000980_004.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Healthy/00001027_001.png",
    ],
    "Infiltration": [
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00000125_000.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00000181_009.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00000376_010.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00000506_003.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00000548_001.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00000652_012.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00000796_009.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00000873_000.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00001001_002.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Infiltration/00001075_005.png",
    ],
    "Nodule": [
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00000105_004.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00000157_000.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00000250_003.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00000336_000.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00000415_000.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00000459_017.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00000634_004.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00000640_001.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00000744_001.png",
        "/content/drive/MyDrive/Machine-learning-project/Xception/image/Nodule/00001012_000.png",
    ],
}

save_dirs = {
    "Atelectasis": "/content/drive/MyDrive/Machine-learning-project/Xception/image/grad_cam_output_model1/M/Atelectasis",
    "Effusion": "/content/drive/MyDrive/Machine-learning-project/Xception/image/grad_cam_output_model1/M/Effusion",
    "Healthy": "/content/drive/MyDrive/Machine-learning-project/Xception/image/grad_cam_output_model1/M/Healthy",
    "Infiltration": "/content/drive/MyDrive/Machine-learning-project/Xception/image/grad_cam_output_model1/M/Infiltration",
    "Nodule": "/content/drive/MyDrive/Machine-learning-project/Xception/image/grad_cam_output_model1/M/Nodule",
}

if __name__ == "__main__":
    model = densenet121(pretrained=True)
    target_layer = model.features.denseblock4.denselayer16.conv2

    for class_name, img_list in image_paths.items():
        save_dir = save_dirs[class_name]
        for img_path in img_list:
            output_save_path = os.path.join(save_dir, os.path.basename(img_path))
            process_and_save_grad_cam(model, img_path, target_layer, output_save_path)


**Confusion matrix**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

csv_path = '/content/drive/MyDrive/ML/data/valid_young.csv'
df = pd.read_csv(csv_path)

true_labels = df.iloc[:, 1:].values

probs = np.load('/content/drive/MyDrive/ML/history_model/CheXneXt_age/young/ep_10/predict/young/predictions/1732372765129/0.927858775074568-valid_young.npy', allow_pickle=True)

class_names = df.columns[1:]

threshold_F = [0.18225671, 0.354695, 0.39824185, 0.3147227]
threshold_M = [0.16655189, 0.34565434, 0.31126136, 0.2879935]
threshold_young = [0.38284844, 0.46058702, 0.4600233, 0.4011343]
threshold_middle = [0.15721625, 0.23821387, 0.3097358, 0.30881613]
threshold_old = [0.32799268, 0.3056149, 0.42771035, 0.43798673]

def plot_confusion_matrix(true_labels, probs, class_names, threshold):
    #plt.figure(figsize=(10, 8))
    pred = np.zeros(probs.shape)
    con_mat = np.zeros((len(class_names), 4))

    for i in range(len(class_names)):
        for j in range(probs[:, i].shape[0]):
          if probs[j, i] >= threshold[i]:
            pred[j, i] = 1
          else:
            pred[j, i] = 0

        cm = confusion_matrix(true_labels[:, i], pred[:, i])
        con_mat[i] = cm.ravel() # tn, fp, fn, tp
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Negative', 'Positive'])
        disp.plot(cmap=plt.cm.Blues)
        plt.title('Confusion Matrix: {}'.format(class_names[i]))
        plt.ylabel('Actual')
        plt.xlabel('Predicted')

    cm_total = confusion_matrix(true_labels.ravel(), pred.ravel())
    disp_total = ConfusionMatrixDisplay(confusion_matrix=cm_total, display_labels=['Negative', 'Positive'])
    disp_total.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix: Total')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')

    return con_mat

con_mat = plot_confusion_matrix(true_labels, probs, class_names, threshold_young)
print("{:20}{:20}{:20}{:20}{:20}".format('Label', 'True_Negative', 'False_Positive', 'False_Negative', 'True_Positive'))
for i in range(4):
  print("{:20}{:20}{:20}{:20}{:20}".format(class_names[i], int(con_mat[i, 0]), int(con_mat[i, 1]), int(con_mat[i, 2]), int(con_mat[i, 3])))
#print(con_mat)
#print(class_names)

Get best model

In [ ]:
from __future__ import print_function
import glob
import os
import sys
import argparse
import json
import itertools

def print_helper(seq, verbose, n_print, reverse=False):
    for i, model_info in enumerate(sorted(seq, reverse=reverse)):
        if verbose:
            print(model_info)
        else:
            print(model_info[1], end=" ")
        if i == n_print:
            break


def get_best_predictions(path, n_print, verbose):
    predictions = []
    for timestamp in glob.glob(path + "/*"):
        for validation_file in glob.glob(timestamp + "/*-valid_1.npy"):
            basename = os.path.basename(validation_file)
            dirname = os.path.dirname(validation_file)
            with open(dirname + '/params.json') as json_data:
                params = json.load(json_data)
                num_models = params["num_models"]
            end_index = basename.find("-valid_1")
            AUC = float(basename[:end_index])
            predictions.append((AUC, dirname, num_models))
    print_helper(predictions, verbose, n_print, reverse=True)


def get_best_models(path, n, verbose, ungrouped):
    # from https://stackoverflow.com/questions/3368969/find-string-between-two-substrings
    def find_between(s, first, last):
        try:
            start = s.index(first ) + len( first )
            end = s.index( last, start )
            return s[start:end]
        except ValueError:
            return ""

    models = []
    for checkpoint_path in glob.glob(path + "/**/*epoch*"):
        val_loss = float(find_between(
            checkpoint_path, 'val', '_train'))
        models.append((val_loss, checkpoint_path))

    groups = itertools.groupby(models,
                lambda x: os.path.dirname(x[1]))

    models_group_best = []
    if ungrouped is True:
        models_group_best = models
    else:
        for _, model in groups:
            models_group_best.append(sorted(list(model))[0])

    return sorted(models_group_best)[:n]


if __name__ == '__main__':

    """
    parser = argparse.ArgumentParser()
    parser.add_argument(
        'selection_type',
        choices={'model', 'prediction'},
        help="Whether to get the best models or best predictions")
    parser.add_argument(
        'folder',
        help="path to get best from")
    parser.add_argument('-n', '--n', type=int, default=20)
    parser.add_argument('--verbose', action="store_true")
    parser.add_argument('--ungrouped', action="store_true")
    args = parser.parse_args()
    """
    args = argparse.Namespace(
        selection_type="model",
        #selection_type="prediction",
        folder="/content/drive/MyDrive/ML/history_model",
        n=5,
        verbose=False,
        ungrouped=False
    )

    if args.selection_type == 'model':
        models_group_best = get_best_models(args.folder, args.n, args.verbose, args.ungrouped)
        print_helper(models_group_best, args.verbose, args.n, reverse=False)
    elif args.selection_type == 'prediction':
        get_best_predictions(args.folder, args.n, args.verbose)